In [ ]:
# ===== STEP 1: INSTALL REQUIRED PACKAGES =====
!pip install crewai==0.134.0
!pip install 'crewai[tools]'

]) (33.1.0)


In [ ]:
# ===== STEP 2: IMPORT LIBRARIES =====
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ===== STEP 3: SET UP API KEYS & LLMs=====
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [ ]:
# ===== Create LLMs
groq_llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.1
)

gemini_llm = LLM(
    model="gemini/gemini-2.0-flash",
    # api_key=userdata.get('GEMINI_API_KEY'),
    temperature=0.1
)

# openai_llm = LLM(
#     model="openai/gpt-4o-mini",
#     temperature=0.1
# )

In [ ]:
# ===== STEP 4: INITIALIZE TOOLS =====
web_search_tool = SerperDevTool()

In [ ]:
# ===== STEP 5: CREATE AGENTS =====
# Market Research Analyst Agent
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends and emerging technologies. You have years of experience in technology market research and can identify key developments that will shape the future.',
    tools=[web_search_tool],
    verbose=True,  # Fixed typo: was "verbost"
    allow_delegation=False,
    llm=gemini_llm
)

# Content Writer Agent
writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for making complex technology topics accessible to everyone. You excel at transforming technical research into engaging, informative content.',
    tools=[],
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm
)

# LinkedIn Post Creator Agent
linkedin_creator = Agent(
    role='LinkedIn Content Strategist',
    goal='Transform blog content into compelling LinkedIn posts that spark engagement and meaningful conversations',
    backstory="""You are a social media content strategist who creates posts that spark real engagement
    and meaningful conversations. Your writing style is conversational, direct, and actionable - like a
    trusted mentor sharing hard-won insights over coffee. You specialize in the LinkedIn platform and
    understand what makes professionals stop scrolling, engage, and share content. You know how to balance
    authenticity with value, creating posts that feel personal yet professional.""",
    tools=[],
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm,
)

In [ ]:
# ===== STEP 6: CREATE TASKS =====
# Research Task
research = Task(
    description="""Search the web for the latest AI trends and developments. Focus on:

    1. Recent breakthrough technologies or AI models
    2. Major company announcements or partnerships
    3. Regulatory developments or policy changes
    4. Market growth predictions and statistics
    5. Emerging applications or use cases

    Provide a comprehensive summary with specific examples and data where available.""",

    expected_output='A detailed summary of the top 3-5 trending developments in AI with insights on their potential impact on businesses and society',
    agent=researcher
)

# Writing Task
write = Task(
    description="""Write an engaging blog post about the AI industry based on the research analyst's findings.

    The blog post should:
    - Have an attention-grabbing title
    - Include an engaging introduction
    - Cover the main trends identified by the researcher
    - Explain the implications for readers
    - End with a thought-provoking conclusion
    - Be written in an accessible, engaging tone
    - Use markdown formatting for better readability""",

    expected_output="A well-structured, 4-6 paragraph blog post in markdown format with engaging content that makes AI trends accessible to a general audience",
    agent=writer,
    context=[research],  # Added context to link tasks
    output_file='ai_industry_blog_post.md'  # Fixed filename and path
)

# LinkedIn Posts Creation Task
linkedin_posts = Task(
    description="""Transform the blog post content into 3 distinct, highly engaging LinkedIn posts that professionals will want to read, comment on, and share.

    **Create exactly 3 posts with different angles:**

    **POST 1: THE INSIGHT ANGLE**
    - Lead with a counterintuitive truth or surprising finding from the blog
    - Share why this matters in today's business landscape
    - Include a personal reflection or "aha moment"
    - End with a thought-provoking question for engagement

    **POST 2: THE STORY ANGLE**
    - Extract the human story from the blog content
    - Focus on the journey, struggle, or transformation in AI
    - Make it relatable to your audience's challenges
    - Include lessons learned and vulnerability
    - Close with actionable advice

    **POST 3: THE FRAMEWORK ANGLE**
    - Distill the blog's key insights into a simple framework
    - Present it as 3-5 clear, actionable steps
    - Include specific examples or applications
    - Position as a tool readers can immediately use
    - End with encouragement to take action

    **Writing Style Requirements:**
    - Conversational, mentor-like tone
    - 150-300 words per post
    - Short paragraphs (1-2 sentences max)
    - Frequent line breaks for mobile reading
    - Personal and authentic voice
    - Include specific examples and numbers
    - End each post with engaging questions
    - Avoid jargon and corporate speak
    - Use "you" to make it personal""",

    expected_output="""3 distinct LinkedIn posts in the following format:

    ## LinkedIn Post 1: The Insight Angle
    [150-300 word post with counterintuitive insight, personal reflection, and engaging question]

    ## LinkedIn Post 2: The Story Angle
    [150-300 word post with human story, relatable challenges, and actionable advice]

    ## LinkedIn Post 3: The Framework Angle
    [150-300 word post with simple framework, clear steps, and call to action]

    Each post should be ready to copy/paste directly to LinkedIn and optimized for engagement.""",

    agent=linkedin_creator,
    context=[research, write],
    output_file='linkedin_posts.md'
)

In [ ]:
# ===== STEP 7: CREATE AND RUN CREW =====
crew = Crew(
    agents=[researcher, writer, linkedin_creator],
    tasks=[research, write, linkedin_posts],
    verbose=True,
)


result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a1ab0c5b-9458-499d-a149-656d36fa0662                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Search the web for the latest AI trends and developments. Focus on:                                      │
│                                                                                                                 │
│      1. Recent breakthrough technologies or AI models                                                           │
│      2. Major company announcements or partnerships                                                             │
│      3. Regulatory developments or policy changes                                                               │
│      4. Market growth predictions and statistics                                                                │
│      5. Emerging applications or use cases                                                                      │
│                                                                                                                 │
│      Provide a comprehensive summary with specific examples and data where available.                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Okay, I understand. I need to use the search tool to find the latest AI trends and developments,      │
│  focusing on the specified areas. I will then compile a comprehensive summary of the top 3-5 trends with        │
│  insights on their potential impact.                                                                            │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"latest AI trends and developments\"}"                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: LinkedIn Content Strategist                                                                             │
│                                                                                                                 │
│  Task: Transform the blog post content into 3 distinct, highly engaging LinkedIn posts that professionals will  │
│  want to read, comment on, and share.                                                                           │
│                                                                                                                 │
│      **Create exactly 3 posts with different angles:**                                                          │
│                                                                                                                 │
│      **POST 1: THE INSIGHT ANGLE**                                                                              │
│      - Lead with a counterintuitive truth or surprising finding from the blog                                   │
│      - Share why this matters in today's business landscape                                                     │
│      - Include a personal reflection or "aha moment"                                                            │
│      - End with a thought-provoking question for engagement                                                     │
│                                                                                                                 │
│      **POST 2: THE STORY ANGLE**                                                                                │
│      - Extract the human story from the blog content                                                            │
│      - Focus on the journey, struggle, or transformation in AI                                                  │
│      - Make it relatable to your audience's challenges                                                          │
│      - Include lessons learned and vulnerability                                                                │
│      - Close with actionable advice                                                                             │
│                                                                                                                 │
│      **POST 3: THE FRAMEWORK ANGLE**                                                                            │
│      - Distill the blog's key insights into a simple framework                                                  │
│      - Present it as 3-5 clear, actionable steps                                                                │
│      - Include specific examples or applications                                                                │
│      - Position as a tool readers can immediately use                                                           │
│      - End with encouragement to take action                                                                    │
│                                                                                                                 │
│      **Writing Style Requirements:**                                                                            │
│      - Conversational, mentor-like tone                                                                         │
│      - 150-300 words per post                                                                                   │
│      - Short paragraphs (1-2 sentences max)                                                                     │
│      - Frequent line breaks for mobile reading                                                                  │
│      - Personal and authentic voice                    

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: LinkedIn Content Strategist                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  ## LinkedIn Post 1: The Insight Angle                                                                          │
│                                                                                                                 │
│  Did you know that Small Language Models (SLMs) are poised to disrupt the AI landscape, even with all the buzz  │
│  around LLMs?                                                                                                   │
│                                                                                                                 │
│  It's true! While everyone's focused on massive models, these smaller, more efficient AI engines are gaining    │
│  traction.                                                                                                      │
│                                                                                                                 │
│  Why does this matter? SLMs can run on smaller devices, making AI accessible to more people and reducing        │
│  computational costs. Think edge computing, personalized experiences on your phone, and AI-powered solutions    │
│  in areas with limited resources.                                                                               │
│                                                                                                                 │
│  My "aha" moment came when I realized the democratization potential here. It's not just about big tech; it's    │
│  about empowering individuals and smaller businesses.                                                           │
│                                                                                                                 │
│  What are your thoughts? How do you see SLMs impacting your industry or daily life? #AI #SmallLanguageModels    │
│  #Innovation #EdgeComputing                                                                                     │
│                                                                                                                 │
│  ## LinkedIn Post 2: The Story Angle                                                                            │
│                                                                                                                 │
│  I've been watching the AI regulatory landscape closely, and it reminds me of the early days of the internet.   │
│  Excitement, innovation, but also a growing sense of "we need some rules here."                                 │
│                                                                                                                 │
│  The EU AI Act and Colorado AI Act are just the beginning. We're seeing a global scramble to define ethical     │
│  boundaries and responsible AI development.                                                                     │
│                                                                                                                 │
│  The struggle is real for companies trying to navigate this evolving landscape. Compliance is complex, and the  │
│  potential for stifled innovation is a valid concern.                                                           │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d6ecce87-cee1-4314-ac42-a010172bf7df                                                                     │
│  Agent: LinkedIn Content Strategist                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a1ab0c5b-9458-499d-a149-656d36fa0662                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```                                                                                              │
│  ## LinkedIn Post 1: The Insight Angle                                                                          │
│                                                                                                                 │
│  Did you know that Small Language Models (SLMs) are poised to disrupt the AI landscape, even with all the buzz  │
│  around LLMs?                                                                                                   │
│                                                                                                                 │
│  It's true! While everyone's focused on massive models, these smaller, more efficient AI engines are gaining    │
│  traction.                                                                                                      │
│                                                                                                                 │
│  Why does this matter? SLMs can run on smaller devices, making AI accessible to more people and reducing        │
│  computational costs. Think edge computing, personalized experiences on your phone, and AI-powered solutions    │
│  in areas with limited resources.                                                                               │
│                                                                                                                 │
│  My "aha" moment came when I realized the democratization potential here. It's not just about big tech; it's    │
│  about empowering individuals and smaller businesses.                                                           │
│                                                                                                                 │
│  What are your thoughts? How do you see SLMs impacting your industry or daily life? #AI #SmallLanguageModels    │
│  #Innovation #EdgeComputing                                                                                     │
│                                                                                                                 │
│  ## LinkedIn Post 2: The Story Angle                                                                            │
│                                                                                                                 │
│  I've been watching the AI regulatory landscape closely, and it reminds me of the early days of the internet.   │
│  Excitement, innovation, but also a growing sense of "we need some rules here."                                 │
│                                                                                                                 │
│  The EU AI Act and Colorado AI Act are just the beginning. We're seeing a global scramble to define ethical     │
│  boundaries and responsible AI development.                                                                     │
│                                                                                                                 │
│  The struggle is real for companies trying to navigate this evolving landscape. Compliance is complex, and the  │
│  potential for stifled innovation is a valid concern. 